<b>UAZ 469 Army SUV Frame modeling: FEM </b>

<i><b>Tsybakov Mikhail, Dec. 2023</b> </i>

In [1]:
# All needed imports
import meshio
import numpy as np 
import pyvista as pv
from scipy import sparse
from scipy.sparse.linalg import spsolve
from matplotlib import pyplot as plt

from time import time
import warnings
warnings.filterwarnings('ignore')

In [31]:
# Mesh visualization

def visualize(grid, scalars_ = None, cmap_ = None):    
    pv.set_jupyter_backend('client')
    plotter = pv.Plotter(lighting = 'three lights')
    
    if scalars_ is None:
        plotter.add_mesh(grid, show_edges = True)
    else:
        plotter.add_mesh(grid, show_edges = False, scalars = scalars_, cmap = cmap_)
    
    plotter.set_background('white')
    plotter.show_bounds(color = 'black')
    plotter.add_bounding_box(color = 'black')
    plotter.show()

In [3]:
# Reading mesh

def read_mesh(filename):
    mesh = meshio.read(filename)
    
    nodes = mesh.points
    elems = mesh.cells[0].data
    
    print(f'Nodes count: {nodes.shape[0]}')
    print(f'Elems count: {len(elems)}')
    
    return nodes, elems

In [4]:
# Converting mesh to PyVista unstructured grid

def mesh_to_unstructured_grid(nodes, elems):
    grid_points    = nodes
    grid_cells     = []
    grid_celltypes = []
    
    for elem in elems:
        grid_cells += ([4] + list(elem))
        grid_celltypes.append(pv.CellType.TETRA)
        
    grid = pv.UnstructuredGrid(grid_cells, grid_celltypes, grid_points)
    return grid

In [5]:
# Local stiffness matrix for 1st order 3D tetraedron element

def local_K(elem_id):
    e_nodes = tuple(elems[elem_id])
    
    x1, y1, z1 = nodes[e_nodes[0]]
    x2, y2, z2 = nodes[e_nodes[1]]
    x3, y3, z3 = nodes[e_nodes[2]]
    x4, y4, z4 = nodes[e_nodes[3]]
    
    jacobian = np.array([
        [x2 - x1, y2 - y1, z2 - z1],
        [x3 - x1, y3 - y1, z3 - z1],
        [x4 - x1, y4 - y1, z4 - z1]
    ])
    
    volume_matrix = np.array([
        [1, x1, y1, z1],
        [1, x2, y2, z2],
        [1, x3, y3, z3],
        [1, x4, y4, z4]
    ])
    
    inv_jacobian = np.linalg.inv(jacobian)
    
    a, b, c = inv_jacobian[0, 0], inv_jacobian[0, 1], inv_jacobian[0, 2]
    d, e, f = inv_jacobian[1, 0], inv_jacobian[1, 1], inv_jacobian[1, 2]
    g, h, k = inv_jacobian[2, 0], inv_jacobian[2, 1], inv_jacobian[2, 2]
    
    N1_block_top = np.array([
        [-a-b-c, 0, 0],
        [0, -d-e-f, 0],
        [0, 0, -g-h-k]
    ])
    
    N1_block_bottom = np.array([
        [-d-e-f, -a-b-c, 0],
        [0, -g-h-k, -d-e-f],
        [-g-h-k, 0, -a-b-c],
    ])
    
    N2_block_top = np.array([
        [a, 0, 0],
        [0, d, 0],
        [0, 0, g]
    ])
    
    N2_block_bottom = np.array([
        [d, a, 0],
        [0, g, d],
        [g, 0, a]
    ])
    
    N3_block_top = np.array([
        [b, 0, 0],
        [0, e, 0],
        [0, 0, h]
    ])
    
    N3_block_bottom = np.array([
        [e, b, 0],
        [0, h, e],
        [h, 0, b]
    ])
    
    N4_block_top = np.array([
        [c, 0, 0],
        [0, f, 0],
        [0, 0, k]
    ])
    
    N4_block_bottom = np.array([
        [f, c, 0],
        [0, k, f],
        [k, 0, c]
    ])
    
    B_top    = np.concatenate([N1_block_top, N2_block_top, N3_block_top, N4_block_top], axis = 1)
    B_bottom = np.concatenate([N1_block_bottom, N2_block_bottom, N3_block_bottom, N4_block_bottom], axis = 1)
    
    B = np.concatenate([B_top, B_bottom], axis = 0)
    
    tetra_volume = np.abs(np.linalg.det(jacobian))
    
    local_K = (1/6) * B.T @ D @ B * np.abs(np.linalg.det(volume_matrix))
    return local_K, B

In [6]:
# For each element, calculates degrees of freedom, related to it

def map_dofs(elements):
    dofs_mapping = np.zeros((n_elems, 12))
    for i, elem in enumerate(elements):
        dofs = []
        for node in elem:
            dofs += [node*3, node*3 + 1, node*3 + 2]
        dofs_mapping[i] = np.array(dofs)
    return dofs_mapping

In [7]:
def mesh_bounds(nodes):
    nodes_np = np.array(nodes)
    
    min_x = np.min(nodes_np[:, 0])
    max_x = np.max(nodes_np[:, 0])
    min_y = np.min(nodes_np[:, 1])
    max_y = np.max(nodes_np[:, 1])
    min_z = np.min(nodes_np[:, 2])
    max_z = np.max(nodes_np[:, 2])
    
    return min_x, max_x, min_y, max_y, min_z, max_z

In [8]:
# Founds bounding nodes with tolerance

def bounding_nodes(bounds, tol):
    min_x, max_x, min_y, max_y, min_z, max_z = bounds
    
    x0_nodes = []
    x1_nodes = []
    y0_nodes = []
    y1_nodes = []
    z0_nodes = []
    z1_nodes = []
    
    for i, node in enumerate(nodes):
        x, y, z = tuple(node)
        if abs(x - min_x) < tol:
            x0_nodes.append(i)
        if abs(x - max_x) < tol:
            x1_nodes.append(i)
        if abs(y - min_y) < tol:
            y0_nodes.append(i)
        if abs(y - max_y) < tol:
            y1_nodes.append(i)
        if abs(z - min_z) < tol:
            z0_nodes.append(i)
        if abs(z - max_z) < tol:
            z1_nodes.append(i)
            
    return x0_nodes, x1_nodes, y0_nodes, y1_nodes, z0_nodes, z1_nodes

In [9]:
# Founds middle part of the frame

def find_middle(x0, x1, tol):
    min_x = x0
    max_x = x1
    
    min_y = 1e9
    max_y = -1e9
    min_z = 1e9
    max_z = -1e9
    
    upper_nodes_mid = []
    
    for node in nodes:
        x,y,z = tuple(node)
        if (x >= min_x and x <= max_x):
            if (y < min_y):
                min_y = y
            if (y > max_y):
                max_y = y
            if (z < min_z):
                min_z = z
            if (z > max_z):
                max_z = z
                
    for i, node in enumerate(nodes):
        x,y,z = tuple(node)
        if abs(y - max_y) < tol and x >= min_x and x <= max_x:
            upper_nodes_mid.append(i)
            
    return upper_nodes_mid

In [10]:
# Global stiffness matrix assembly

def assemble_K(elements, dofs_mapping):
    values = 12*12
    global_iK = np.zeros(values * n_elems)
    global_jK = np.zeros(values * n_elems)
    global_aK = np.zeros(values * n_elems)

    for ei, elem in enumerate(elements):
        iK = np.kron(dofs_mapping[ei], np.ones(12))
        jK = np.reshape(np.kron(dofs_mapping[ei], np.reshape(np.ones(12), (12,1))), -1)
        local_K_, B_ = local_K(ei)
        aK = np.reshape(local_K_, -1)
        
        elems_data[ei]['B_matr'] = B_
        
        global_iK[values*ei : values*(ei+1)] = iK
        global_jK[values*ei : values*(ei+1)] = jK
        global_aK[values*ei : values*(ei+1)] = aK

    global_K = sparse.coo_matrix((global_aK,(global_iK, global_jK)),shape = (n_dofs, n_dofs)).tocsr()
    return global_K

In [11]:
# Fixes specified node

def fix_node(node_id, fix_X, fix_Y, fix_Z):
    dof_x = node_id*3
    dof_y = node_id*3 + 1
    dof_z = node_id*3 + 2
    
    if (fix_X):
        fixed_dofs.append(dof_x)
    if (fix_Y):
        fixed_dofs.append(dof_y)
    if (fix_Z):
        fixed_dofs.append(dof_z)
        
# Applies load

def apply_load(node_id, load_X, load_Y, load_Z):
    dof_x = node_id*3
    dof_y = node_id*3 + 1
    dof_z = node_id*3 + 2
    
    F_global[dof_x] = load_X
    F_global[dof_y] = load_Y
    F_global[dof_z] = load_Z

In [12]:
# Reading mesh

filename     = 'UAZ_469_Frame_1_tetra.inp'
nodes, elems = read_mesh(filename)

Nodes count: 28100
Elems count: 105171


In [13]:
# Visualisation

pv_grid = mesh_to_unstructured_grid(nodes, elems)
visualize(pv_grid)

Widget(value="<iframe src='http://localhost:51392/index.html?ui=P_0x235b46152e0_0&reconnect=auto' style='width…

In [14]:
# Analysis setup
# * * * * * * * * * * * * * * * * * * * * * * * *
# Isotropic steel is considered as frame material

E  = 210*1e9
nu = 0.3

n_nodes      = nodes.shape[0]
n_elems      = len(elems)
n_dofs       = n_nodes * 3 
dofs_mapping = map_dofs(elems)

K_global = None
F_global = np.zeros(n_dofs)
U_global = np.zeros(n_dofs)

all_dofs   = np.arange(n_dofs)
fixed_dofs = []

elems_data = {
    elem_id: {
        'stress': None,
        'strain': None,
        'B_matr': None
    } for elem_id in range(len(elems))
}

# 3D Isotropic Hook's law in matrix form:
# * * * * * * * * * * * * * * * * * * * * 
# Warning: not multiplying by E due to high condition number of matrix in this case
# Dividing the displacements by E later due to problem linearity

multiplier = 1/((1 + nu)*(1 - 2*nu))
D = multiplier * np.array([
    [1-nu, nu,   nu,   0,          0,                   0],
    [nu,   1-nu, nu,   0,          0,                   0],
    [nu,   nu,   1-nu, 0,          0,                   0],
    [0,    0,    0,    (1-2*nu)/2, 0,                   0],
    [0,    0,    0,    0,          (1-2*nu)/2,          0],
    [0,    0,    0,    0,          0,          (1-2*nu)/2]
])

In [15]:
# Global stiffness matrix assembly

start_tick = time()
K_global   = assemble_K(elems, dofs_mapping)
end_tick   = time()

print(f'Global stiffness matrix shape: \t({n_dofs} x {n_dofs})')
print(f'Assembly taken: \t\t{end_tick - start_tick :.3f} sec')

Global stiffness matrix shape: 	(84300 x 84300)
Assembly taken: 		16.587 sec


In [16]:
# Mesh sizes

min_x, max_x, min_y, max_y, min_z, max_z = mesh_bounds(nodes)
delta_x = max_x - min_x
delta_y = max_y - min_y
delta_z = max_z - min_z

print(f'X size: {delta_x :.3f}')
print(f'Y size: {delta_y :.3f}')
print(f'Z size: {delta_z :.3f}')

print(f'Min x: {min_x :.3f}')
print(f'Max x: {max_x :.3f}')

X size: 12292.739
Y size: 646.867
Z size: 3752.339
Min x: -6376.642
Max x: 5916.097


In [17]:
# Getting boundary nodes

glob_bounds = (min_x, max_x, min_y, max_y, min_z, max_z)
global_boundaries = bounding_nodes(glob_bounds, tol = 40)
glob_min_x, glob_max_x, glob_min_y, glob_max_y, glob_min_z, glob_max_z = global_boundaries

# Getting middle boundaries

mid_top_3 = find_middle(1800, 2100, 100)
mid_top_2 = find_middle(-150, 250, 10)
mid_top_1 = find_middle(-5800, -5500, 20)


In [18]:
# Drawing fixed nodes

scalars = np.zeros(n_nodes)
for node_ in glob_min_x:
    scalars[node_] = 1
for node_ in glob_max_x:
    scalars[node_] = 1
    
cmap = 'ocean'
#visualize(pv_grid, scalars, cmap)

In [19]:
# Drawing loaded nodes

scalars = np.zeros(n_nodes)
for node_ in mid_top_3:
    scalars[node_] = 1
    
for node_ in mid_top_2:
    scalars[node_] = 1
    
for node_ in mid_top_1:
    scalars[node_] = 1
    
cmap = 'Reds'
#visualize(pv_grid, scalars, cmap)

In [20]:
# Fixing chassis and applying loads from hull:

force = 1e1

for node_ in glob_min_x:
    fix_node(node_, True, True, True)
for node_ in glob_max_x:
    fix_node(node_, True, True, True)
    
for node_ in mid_top_1:
    apply_load(node_, 0, -force, 0)
for node_ in mid_top_2:
    apply_load(node_, 0, -2*force, 0)
for node_ in mid_top_3:
    apply_load(node_, 0, -force, 0)

In [21]:
# Solving

start_tick = time()

free_dofs = np.setdiff1d(all_dofs, fixed_dofs)
U_global[free_dofs] = spsolve(K_global[free_dofs, :][:, free_dofs], F_global[free_dofs])

end_tick   = time()
print(f'Solution taken: {end_tick - start_tick :.3f} sec')

Solution taken: 1.960 sec


In [45]:
nodal_stresses = {
    ni: [] for ni in range(n_nodes)
}

# Calculating strains
for ei in range(len(elems)):
    B_matr = elems_data[ei]['B_matr']
    u_vec  = U_global[dofs_mapping[ei].astype(np.int32)]
    strain = np.dot(B_matr, u_vec)
    stress = np.dot(D, strain)
    elems_data[ei]['strain'] = strain
    elems_data[ei]['stress'] = stress
    
    for node in elems[ei]:
        if nodes[node][0] > -6000 and nodes[node][0] < 5500:
            nodal_stresses[node].append(stress[1])
        else:
            nodal_stresses[node].append(0)

In [46]:
# Averaging
nodal_stresses_avg = []
for ns in nodal_stresses:
    nodal_stresses_avg.append(np.mean(np.array(nodal_stresses[ns])))

In [48]:
visualize(pv_grid, nodal_stresses_avg, "turbo")

Widget(value="<iframe src='http://localhost:51392/index.html?ui=P_0x235b4ce2be0_0&reconnect=auto' style='width…